In [28]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [29]:
from sklearn.metrics import mean_squared_error

# LSTM FORECASTING SETUP

Defining methods to set up data to return sequences of varying lengths (n_steps) for x and y:


We also plan to try to clean up the data more exhaustively.

In [46]:
LSTMData = pd.read_csv("/home/jik19004/FilesToRun/DINGRESEARCH/DingResearchDec28th/aggregatedWeatherData.csv")
display(LSTMData)

,DateTime,Demand_x,HVN_station,HVN_valid,HVN_lon,HVN_lat,HVN_elevation,HVN_tmpf,HVN_dwpf,HVN_relh,...,MMK_dwpf,MMK_relh,MMK_drct,MMK_sknt,MMK_alti,MMK_mslp,MMK_vsby,MMK_skyl1,MMK_feel,MMK_metar
0,2011-01-01 00:00:00,3053.00,HVN,2011-01-01 00:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,...,26.06,88.82,0.0,0.0,30.11,1019.700000,4.0,0.000000,0.000000,KMMK 010553Z AUTO 00000KT 4SM BR CLR M02/M03 A...
1,2011-01-01 01:00:00,2892.00,HVN,2011-01-01 01:53:00,-72.8868,41.2638,4.0,37.940000,32.000000,78.960000,...,26.06,88.82,0.0,0.0,30.10,1019.600000,4.0,0.000000,0.000000,KMMK 010653Z AUTO 00000KT 4SM BR CLR M02/M03 A...
2,2011-01-01 02:00:00,2774.00,HVN,2011-01-01 02:53:00,-72.8868,41.2638,4.0,37.040000,32.000000,81.800000,...,28.04,81.50,150.0,5.0,30.11,1019.600000,5.0,0.000000,27.680000,KMMK 010753Z AUTO 15005KT 5SM HZ CLR 01/M02 A3...
3,2011-01-01 03:00:00,2710.00,HVN,2011-01-01 03:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,...,26.96,77.96,150.0,4.0,30.11,1019.600000,6.0,0.000000,28.680000,KMMK 010853Z AUTO 15004KT 6SM HZ CLR 01/M03 A3...
4,2011-01-01 04:00:00,2698.00,HVN,2011-01-01 04:53:00,-72.8868,41.2638,4.0,30.920000,28.040000,88.920000,...,26.06,88.82,100.0,3.0,30.10,1019.400000,5.0,0.000000,18.786667,KMMK 010953Z AUTO 00000KT 5SM BR CLR M02/M03 A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,2021-12-31 19:00:00,3310.60,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96423,2021-12-31 20:00:00,3148.00,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96424,2021-12-31 21:00:00,2988.93,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96425,2021-12-31 22:00:00,2793.70,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN


In [51]:
print(LSTMData.iloc[61362,:])

DateTime                                     2017-12-31 23:00:00
Demand_x                                                4019.408
HVN_station                                                  HVN
HVN_valid                                    2017-12-31 23:53:00
HVN_lon                                                 -72.8868
                                     ...                        
MMK_mslp                                                  1025.3
MMK_vsby                                                    10.0
MMK_skyl1                                            4433.333333
MMK_feel                                                   -7.89
MMK_metar      KMMK 010453Z AUTO 30003KT 10SM CLR M18/M22 A30...
Name: 61362, Length: 128, dtype: object


Convert the DateTime into integer later

In [31]:
from datetime import datetime 
dateList = [] 
dateTimeCol = LSTMData.loc[:, "DateTime"]
for i in dateTimeCol: 
    x = i.replace("-", "/")
    x = datetime.strptime(x,"%Y/%m/%d %H:%M:%S")
    dateList.append(x)




In [32]:
LSTMData.drop(["DateTime"], axis = 1, inplace= True)
LSTMData.insert(0,"DateTime", dateList, True)
display(LSTMData)

,DateTime,Demand_x,HVN_station,HVN_valid,HVN_lon,HVN_lat,HVN_elevation,HVN_tmpf,HVN_dwpf,HVN_relh,...,MMK_dwpf,MMK_relh,MMK_drct,MMK_sknt,MMK_alti,MMK_mslp,MMK_vsby,MMK_skyl1,MMK_feel,MMK_metar
0,2011-01-01 00:00:00,3053.00,HVN,2011-01-01 00:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,...,26.06,88.82,0.0,0.0,30.11,1019.700000,4.0,0.000000,0.000000,KMMK 010553Z AUTO 00000KT 4SM BR CLR M02/M03 A...
1,2011-01-01 01:00:00,2892.00,HVN,2011-01-01 01:53:00,-72.8868,41.2638,4.0,37.940000,32.000000,78.960000,...,26.06,88.82,0.0,0.0,30.10,1019.600000,4.0,0.000000,0.000000,KMMK 010653Z AUTO 00000KT 4SM BR CLR M02/M03 A...
2,2011-01-01 02:00:00,2774.00,HVN,2011-01-01 02:53:00,-72.8868,41.2638,4.0,37.040000,32.000000,81.800000,...,28.04,81.50,150.0,5.0,30.11,1019.600000,5.0,0.000000,27.680000,KMMK 010753Z AUTO 15005KT 5SM HZ CLR 01/M02 A3...
3,2011-01-01 03:00:00,2710.00,HVN,2011-01-01 03:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,...,26.96,77.96,150.0,4.0,30.11,1019.600000,6.0,0.000000,28.680000,KMMK 010853Z AUTO 15004KT 6SM HZ CLR 01/M03 A3...
4,2011-01-01 04:00:00,2698.00,HVN,2011-01-01 04:53:00,-72.8868,41.2638,4.0,30.920000,28.040000,88.920000,...,26.06,88.82,100.0,3.0,30.10,1019.400000,5.0,0.000000,18.786667,KMMK 010953Z AUTO 00000KT 5SM BR CLR M02/M03 A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,2021-12-31 19:00:00,3310.60,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96423,2021-12-31 20:00:00,3148.00,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96424,2021-12-31 21:00:00,2988.93,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96425,2021-12-31 22:00:00,2793.70,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN


# Converting the datetime into a number: 

In [33]:
for i in range(len(LSTMData.loc[:, "DateTime"])):
    LSTMData.loc[i, "DateTime"] = int(LSTMData.iloc[i,0].strftime("%Y%m%d%H%M%S"))

display(LSTMData.loc[:, "DateTime"])
    

0        20110101000000
1        20110101010000
2        20110101020000
3        20110101030000
4        20110101040000
              ...      
96422    20211231190000
96423    20211231200000
96424    20211231210000
96425    20211231220000
96426    20211231230000
Name: DateTime, Length: 96427, dtype: object

In [34]:
DemandData = LSTMData.loc[:, "Demand_x"].copy()
LSTMData.drop(labels = "Demand_x", axis = 1, inplace= True)
display(LSTMData)

,DateTime,HVN_station,HVN_valid,HVN_lon,HVN_lat,HVN_elevation,HVN_tmpf,HVN_dwpf,HVN_relh,HVN_drct,...,MMK_dwpf,MMK_relh,MMK_drct,MMK_sknt,MMK_alti,MMK_mslp,MMK_vsby,MMK_skyl1,MMK_feel,MMK_metar
0,20110101000000,HVN,2011-01-01 00:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,190.000000,...,26.06,88.82,0.0,0.0,30.11,1019.700000,4.0,0.000000,0.000000,KMMK 010553Z AUTO 00000KT 4SM BR CLR M02/M03 A...
1,20110101010000,HVN,2011-01-01 01:53:00,-72.8868,41.2638,4.0,37.940000,32.000000,78.960000,63.333333,...,26.06,88.82,0.0,0.0,30.10,1019.600000,4.0,0.000000,0.000000,KMMK 010653Z AUTO 00000KT 4SM BR CLR M02/M03 A...
2,20110101020000,HVN,2011-01-01 02:53:00,-72.8868,41.2638,4.0,37.040000,32.000000,81.800000,200.000000,...,28.04,81.50,150.0,5.0,30.11,1019.600000,5.0,0.000000,27.680000,KMMK 010753Z AUTO 15005KT 5SM HZ CLR 01/M02 A3...
3,20110101030000,HVN,2011-01-01 03:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,130.000000,...,26.96,77.96,150.0,4.0,30.11,1019.600000,6.0,0.000000,28.680000,KMMK 010853Z AUTO 15004KT 6SM HZ CLR 01/M03 A3...
4,20110101040000,HVN,2011-01-01 04:53:00,-72.8868,41.2638,4.0,30.920000,28.040000,88.920000,130.000000,...,26.06,88.82,100.0,3.0,30.10,1019.400000,5.0,0.000000,18.786667,KMMK 010953Z AUTO 00000KT 5SM BR CLR M02/M03 A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,20211231190000,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96423,20211231200000,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96424,20211231210000,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96425,20211231220000,NaN,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN


# Filling the NaN values of the stations: 

In [35]:
def notNumericData(data):
    nonNumericColumns = []
    for i in range(len(data.columns)):
        if data.iloc[:,i].dtype != "float64" and data.iloc[:,i].dtype != "int64":
            nonNumericColumns.append(data.columns[i])
    return nonNumericColumns


nonNumericColumns = notNumericData(LSTMData)
nonNumericColumns = nonNumericColumns[1:]
print(nonNumericColumns)

['HVN_station', 'HVN_valid', 'HVN_metar', 'IJD_station', 'IJD_valid', 'IJD_metar', 'BDL_station', 'BDL_valid', 'BDL_metar', 'HFD_station', 'HFD_valid', 'HFD_metar', 'BDR_station', 'BDR_valid', 'BDR_metar', 'GON_station', 'GON_valid', 'GON_metar', 'DXR_station', 'DXR_metar', 'MMK_station', 'MMK_valid', 'MMK_metar']


In [36]:
stationCols = ["HVN_station", "IJD_station", "BDL_station", "HFD_station", "BDR_station", "GON_station", "DXR_station", "MMK_station"] 
def fillStationCols(data): 
    for i in stationCols: 
        dataCol = data.loc[:, i]
        for j in range(len(dataCol)):
            if data.loc[j,i] != i[0:3]:
                data.loc[j,i] = 0 
            else:
                data.loc[j,i] = 1

fillStationCols(LSTMData)
display(LSTMData)


,DateTime,HVN_station,HVN_valid,HVN_lon,HVN_lat,HVN_elevation,HVN_tmpf,HVN_dwpf,HVN_relh,HVN_drct,...,MMK_dwpf,MMK_relh,MMK_drct,MMK_sknt,MMK_alti,MMK_mslp,MMK_vsby,MMK_skyl1,MMK_feel,MMK_metar
0,20110101000000,1,2011-01-01 00:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,190.000000,...,26.06,88.82,0.0,0.0,30.11,1019.700000,4.0,0.000000,0.000000,KMMK 010553Z AUTO 00000KT 4SM BR CLR M02/M03 A...
1,20110101010000,1,2011-01-01 01:53:00,-72.8868,41.2638,4.0,37.940000,32.000000,78.960000,63.333333,...,26.06,88.82,0.0,0.0,30.10,1019.600000,4.0,0.000000,0.000000,KMMK 010653Z AUTO 00000KT 4SM BR CLR M02/M03 A...
2,20110101020000,1,2011-01-01 02:53:00,-72.8868,41.2638,4.0,37.040000,32.000000,81.800000,200.000000,...,28.04,81.50,150.0,5.0,30.11,1019.600000,5.0,0.000000,27.680000,KMMK 010753Z AUTO 15005KT 5SM HZ CLR 01/M02 A3...
3,20110101030000,1,2011-01-01 03:53:00,-72.8868,41.2638,4.0,33.980000,30.020000,85.240000,130.000000,...,26.96,77.96,150.0,4.0,30.11,1019.600000,6.0,0.000000,28.680000,KMMK 010853Z AUTO 15004KT 6SM HZ CLR 01/M03 A3...
4,20110101040000,1,2011-01-01 04:53:00,-72.8868,41.2638,4.0,30.920000,28.040000,88.920000,130.000000,...,26.06,88.82,100.0,3.0,30.10,1019.400000,5.0,0.000000,18.786667,KMMK 010953Z AUTO 00000KT 5SM BR CLR M02/M03 A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,20211231190000,0,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96423,20211231200000,0,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96424,20211231210000,0,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN
96425,20211231220000,0,NaN,-72.8868,41.2638,4.0,46.133333,44.266667,93.153333,86.666667,...,41.00,92.82,0.0,0.0,29.94,1013.833333,4.0,733.333333,43.000000,NaN


In [37]:
print(nonNumericColumns)

['HVN_station', 'HVN_valid', 'HVN_metar', 'IJD_station', 'IJD_valid', 'IJD_metar', 'BDL_station', 'BDL_valid', 'BDL_metar', 'HFD_station', 'HFD_valid', 'HFD_metar', 'BDR_station', 'BDR_valid', 'BDR_metar', 'GON_station', 'GON_valid', 'GON_metar', 'DXR_station', 'DXR_metar', 'MMK_station', 'MMK_valid', 'MMK_metar']


In [38]:
LSTMData.drop(["HVN_valid","HVN_metar", "IJD_valid", "IJD_metar", "BDL_valid", "BDL_metar", "HFD_valid", "HFD_metar", "BDR_valid", 
"BDR_metar", "GON_valid", "GON_metar", "DXR_metar", "MMK_valid", "MMK_metar"], axis=1, inplace=True)
print(len(LSTMData.columns))


112


# Splitting the Data into Training, Validation, and Testing! 

In [39]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameter 
import tensorflow as tf 
from tensorflow import keras as keras 


FOR LSTM Use previous data strictly to predict time t. 
Example: t-5,t-4,t-3,t-2,t-1 to predict the t time. 
This makes sense logically because at time t, you will obtain the values on that day but we want to predict ahead of time! 

In [41]:
def return_sequences(data, outputData, input_n_steps, output_n_steps):
    X = list()
    Y = list() 
    length = len(data)
    for i in range(0,length, input_n_steps):
        input_indx = i + input_n_steps
        output_indx = input_indx + output_n_steps 
        if (output_indx > len(data)): # we need to have equally split sequences. 
            break               # The remaining data that cannot fit into a fixed
                                # sequence will immediately be cut!
        else:
            Xsample = data.iloc[i:input_indx, :] # get the previous data
            Ysample = outputData[input_indx:output_indx]
            X.append(Xsample)
            Y.append(Ysample) 
    return (X, Y)

sequences = return_sequences(LSTMData, DemandData,12,6)
Xsequence = sequences[0]
Ysequence = sequences[1]


In [ ]:
def splitData(data, output):
    dataCopy = data.copy()
    outputCopy = output.copy()
    
    TrainingData = data.iloc[:61363, :]
    TrainingOutput = output[:61363]

    RemainingData = data.iloc[61363:, :]
    RemainingOutput = output[61363:]

    ValidationData = RemainingData.iloc[:int(0.5*len(RemainingData)), :]
    ValidationOutput = RemainingOutput[:int(0.5*len(RemainingOutput))]

    TestingData = RemainingData.iloc[int(0.5 * len(RemainingData)):, :]
    TestingOutput = RemainingOutput[int(0.5 * len(RemainingData)):]


    TrainingSequences = return_sequences(TrainingData, TrainingOutput, 12, 6)

    TransformedTrainingData = TrainingSequences[0]
    TransformedTrainingOutput = TrainingSequences[1]

    ValidationSequences = return_sequences(ValidationData, ValidationOutput, 12,6)

    TransformedValidationData = ValidationSequences[0]
    TransformedValidationOutput = ValidationSequences[1]

    TestingSequences = return_sequences(TestingData, TestingOutput, 12,6)

    TransformedTestingData = TestingSequences[0]
    TransformedTestingOutput = TestingSequences[1]



    return (TransformedTrainingData, TransformedTrainingOutput, TransformedValidationData, TransformedValidationOutput, 
    TransformedTestingData, TransformedTestingOutput)
    


# LSTM Model Construction 

In [45]:
from keras.layers import LSTM as LSTM 
from keras.layers import Dense as Dense
from keras_tuner.engine.hyperparameters import HyperParameter 
from keras.regularizers import l2 
from keras_tuner import RandomSearch 

def ModelConstruction(hp):
    Regularization = l2(l2= 0.003)
    model = keras.Sequential() # let's say that we should use the last 12 hours and will spit out 6 hours into the future basically. 
    model.add(keras.layers.Input((12,112))) # the shape will be number of previous values along with number of columns of our input data. 

    model.add(LSTM(units= hp.Int("numUnits", min_value = 64, max_value = 128, step= 32), dropout = 0.2))
    model.add(LSTM(units= hp.Int("numUnits", min_value = 96, max_value = 160, step=32), dropout= 0.2))

    for i in range(hp.Int('layers',1,3)):
        model.add(Dense(units= hp.Int("numUnits", min_value = 32, max_value = 96, step = 32), use_bias= True), kernel_regularizer = Regularization)
        model.add(keras.layers.Dropout(rate = 0.2))
    model.add(Dense(units= 1, kernel_regularizer = Regularization, use_bias=True)) 

    model.compile(loss= "mean_squared_error", optimizer = "rmsprop")

    return model 

tuner = RandomSearch(ModelConstruction, objective = "val_loss", max_trials = 80)

    

2023-01-08 13:27:21.753096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)